# Lab 1/2 EMG Processing

This lab will go into the basics of real-time classification using EMG signals. 

In [ ]:
import pandas as pd
import numpy as np
import scipy
import scipy.signal
import matplotlib.pyplot as plt

In [ ]:
''' If you have pyqt installed, this command will pop out interactive windows for graphs'''
%matplotlib qt

# Myo Data Processing
The following cells are for EMG data processing from the sample file

In [ ]:
# Lets read in our data
import os
directory = 'Data/'
path = 'p18_emg.csv'
myo_df = pd.read_csv(directory + path)
myo_df.columns = myo_df.columns.str.replace(' ', '')
myo_df = myo_df.groupby('Arm').get_group('left') # This only needs to be done if you have two Myos running at the same time
display(myo_df)

## Plot EMG and IMU Channels

In [ ]:
''' Plot Entire EMG signals'''
for channel in range(1,9):
    plt.figure()
    ax = myo_df['EMG_' + str(channel)].plot()
    plt.title('EMG_' + str(channel))
    plt.ylabel('mVolts')
    plt.xlabel('Time')

for channel in ['X', 'Y', 'Z']:
    plt.figure()
    myo_df['Acc_' + channel].plot()
    plt.title('Acc_' + str(channel))
    plt.ylabel('g')
    plt.xlabel('Time')

## Get Descripitive Statistics for Raw Data

In [ ]:
myo_df.describe()

## Rectify the Signal

In [ ]:
rectified_df =myo_df.copy() # make copy of DF

for col in ['EMG_' + str(i) for i in range(1, 9)]:
    rectified_df[col] = rectified_df[col].apply(abs) # applys the absolute function to each channel
    
display(rectified_df)

### Plot the Rectified Signals and Look at the new Means/Stdevs for each EMG Channel

In [ ]:
'''Your Code Here'''

'''Stop Coding Here'''

### Apply a rolling average with a window size of 400 (2 seconds) and look at the new plots
Look into Pandas rolling method (e.g, myo_df.rolling(400).mean())

In [ ]:
'''Your code here'''

'''Stop coding here'''

### Advanced EMG Filtering
Now, we are going to apply a bandpass filter to each EMG channel.

In [ ]:
import scipy as sp
import scipy.signal

def filteremg(emg, low_pass=3, sfreq=200, high_band=20, low_band=95):
    """
    emg: EMG data
    high: high-pass cut off frequency
    low: low-pass cut off frequency
    sfreq: sampling frequency
    """
    # Zero mean emg signal
    emg = emg - emg.mean()
    
    # normalise cut-off frequencies to sampling frequency
    high_band = high_band/(sfreq/2)
    low_band = low_band/(sfreq/2)
    
    
    # create bandpass filter for EMG
    b1, a1 = sp.signal.butter(4, [high_band,low_band], btype='bandpass', analog=True)
    
    # process EMG signal: filter EMG
    emg_filtered = sp.signal.filtfilt(b1, a1, emg)    
    
    # process EMG signal: rectify
    emg_rectified = abs(emg_filtered)
    
    # create lowpass filter and apply to rectified signal to get EMG envelope
    low_pass = low_pass/(sfreq/2)
    b2, a2 = sp.signal.butter(4, low_pass, fs=sfreq, btype='lowpass')
    emg_envelope = sp.signal.lfilter(b2, a2, emg_rectified)
    
    return emg_envelope
    

filt_emg = myo_df.copy()
emg_keys = ['EMG_' + str(i) for i in range(1, 9)]
filt_emg[emg_keys] = filt_emg[emg_keys].apply(filteremg, raw=True)
display(filt_emg)

## Plot Filtered Signals and Get Mean/Stdev

In [ ]:
''' Your code here'''


''' Stop coding here'''

## Power Spectral Density
Now, we are going to look at a PSF plot

In [ ]:
f, Pxx_den = sp.signal.periodogram(filt_emg['EMG_1'], 200)
plt.semilogy(f, Pxx_den)
plt.ylim([1e-7, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

### Plot the PSD for each filtered EMG channel and Find the Max Power

In [ ]:
''' Your Code Here'''

''' Stop Coding Here'''

# Segmenting Data
Using your collected timestamps, we are now going to look at each gesture: rock, paper, scissors

In [ ]:
'''First, we need to set pandas indexes to timestamps'''
myo_df.index = pd.to_datetime(myo_df['Timestamp'], format='%Y-%m-%d %H:%M:%S %f' )
display(myo_df)

In [ ]:
'''Then we can segment data using the indexes'''
start = pd.to_datetime('2019-02-14 14:46:53')
stop  = pd.to_datetime('2019-02-14 14:48:53')
gesture_df = myo_df[start:stop]
display(gesture_df)
''' Notice that we did not need to specify the nano seconds'''

## Reading in XDF
Now we are going to read in from an XDF file

In [ ]:
''' Imports and useful functions'''
import pyxdf

def xdf_to_dataframe(xdf_data):
    ''' Xdf Data should be a list of streams (dictionaries)
        Function returns a dictionary of dataframes, one dataframe per stream'''  
    dataframes = {}
    for stream in xdf_data:
        df = pd.DataFrame()
        data = stream['time_series']
        timestamps = stream['time_stamps']
        df['Time'] = timestamps
        chan_names, units = get_channel_names(stream['info'])
        counts = data.shape[0]
        for series, name, unit in zip(range(data.shape[1]), chan_names, units):
            df[name[0]]  = data[:, series]
            df[name[0] + '_Unit'] = np.repeat(unit, counts)
        
        for item in stream['info']:
            if item not in ['name', 'desc', 'data']:
                try:
                    df[item] = np.repeat(stream['info'][item], counts)
                except:
                    continue
        dataframes[stream['info']['name'][0]] = df
        
    return dataframes
            
        
        

def get_channel_names(info):
    channels = info['desc'][0]['channels'][0]['channel']
    names = [chan['label'] for chan in channels ]
    units = [chan['unit'] for chan in channels ]
    return names, units

In [ ]:
data, header = pyxdf.load_xdf('Data/test.xdf')
dfs = xdf_to_dataframe(data)
display(dfs['BioRadio-20312'])
''' Save new dataframe'''
dfs['BioRadio-20312'].to_csv('Data/test.csv')

# Your turn!
The above code introduced you to some simple EMG processing from already collected data, but now lets use your data.

For the Myo and BioRadio, do the following for each EMG channel:
- Import the files
- Segment the data for one Rock, Paper, and Scissors gesture based on your collected timestamps
- Filter each gesture
- Plot the Filtered EMG signals and the Power Spectral Density of each Gesture
- Determine the Max Power for each Gesture
- Determine the mean/standard deviation for each Gesture

Your report should include each plot in an appendix.

In the main body of your report, include a table for each gesture that has columns: Device, EMG Channel Number, Mean, Std Dev, Max Power. Fill out the rows